Буду работать с данными https://www.kaggle.com/datasets/lokeshparab/amazon-products-dataset/
моя основная задача понять как работает ранжирование и написать простенькую програму локального поискавика по данным

In [2]:
import pandas as pd
import numpy as np
import os

In [16]:
!unzip "/content/archive.zip" -d "/content/data"

Archive:  /content/archive.zip
  inflating: /content/data/Air Conditioners.csv  
  inflating: /content/data/All Appliances.csv  
  inflating: /content/data/All Books.csv  
  inflating: /content/data/All Car and Motorbike Products.csv  
  inflating: /content/data/All Electronics.csv  
  inflating: /content/data/All English.csv  
  inflating: /content/data/All Exercise and Fitness.csv  
  inflating: /content/data/All Grocery and Gourmet Foods.csv  
  inflating: /content/data/All Hindi.csv  
  inflating: /content/data/All Home and Kitchen.csv  
  inflating: /content/data/All Movies and TV Shows.csv  
  inflating: /content/data/All Music.csv  
  inflating: /content/data/All Pet Supplies.csv  
  inflating: /content/data/All Sports Fitness and Outdoors.csv  
  inflating: /content/data/All Video Games.csv  
  inflating: /content/data/Amazon Fashion.csv  
  inflating: /content/data/Amazon Pharmacy.csv  
  inflating: /content/data/Amazon-Products.csv  
  inflating: /content/data/Baby Bath Skin 

In [3]:
folder_path = '/content/data'
df = pd.DataFrame()

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        full_path = os.path.join(folder_path, file)
        data = pd.read_csv(full_path)
        df = pd.concat([df, data], ignore_index=True)

In [4]:
df['discount_price'] = (
    df['discount_price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace(['', 'NaN', 'null', 'None'], np.nan)
    .astype(float)
    .fillna(0)
)

In [5]:
df['actual_price'] = (
    df['actual_price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace(['', 'NaN', 'null', 'None'], np.nan)
    .astype(float)
    .fillna(0)
)

In [7]:
df['ratings'] = (
    df['ratings']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace(['', 'NaN', 'null', 'None', 'Get', 'FREE'], np.nan)
    .astype(float)
    .fillna(0)
)

In [20]:
# df['no_of_ratings'] = (
#     df['no_of_ratings']
#     .str.replace('₹', '', regex=False)
#     .str.replace(',', '', regex=False)
#     .replace(['', 'NaN', 'null', 'None', 'Get', 'FREE', 'FREE Delivery by Amazon', 'Only 1 left in stock.', 'Only 2 left in stock.', 'Usually dispatched in 2 to 3 days.', 'Usually dispatched in 3 to 5 days.', 'Usually dispatched in 4 to 5 days.', 'Usually dispatched in 5 to 6 days.', 'Usually dispatched in 6 to 7 days.', 'Usually dispatched in 7 to 8 days.', 'Usually dispatched in 3 to 4 weeks.'], np.nan)
#     .astype(float)
#     .fillna(0)
# )

ValueError: could not convert string to float: 'Only 3 left in stock.'

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         1103170 non-null  float64
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  1103170 non-null  float64
 8   actual_price    1103170 non-null  float64
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(4), object(6)
memory usage: 84.2+ MB


In [22]:
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Protoner 16Kg PVC Combo Home Gym Package,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71K-gcsnOY...,https://www.amazon.in/Protoner-16Kg-PVC-Combo-...,3.6,917,965.08,4999.0,NaN
1,Unisex Hexagon Rubber Vinyl Fixed Dumbbell - P...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71pcrSXprs...,https://www.amazon.in/Generic-Vinyl-Coated-Iro...,4.4,367,389.00,6450.0,NaN
2,LAFILLETTE 150 lbs Metal Hand Gripper Forearm ...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41oTg0XNbF...,https://www.amazon.in/LALA-LIFE-Gripper-Streng...,4.5,381,489.00,1199.0,NaN
3,Aurion Genuine Leather Pro Weight Lifting Belt...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/813g2hRYuo...,https://www.amazon.in/Genuine-Comfortable-Adju...,4.4,446,799.00,899.0,NaN
4,SPIRO PVC (Round/Hex) Dumbbells (1 Kg. X 2 = 2...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/61VcW8otph...,https://www.amazon.in/Dumbbells-Equipment-Fitn...,3.5,52,82.31,200.0,NaN


In [24]:
from IPython.display import Image, display
from random import randint

for i in range(3):
  index = randint(0, len(df))
  url = df.iloc[index]['image']
  display(Image(url=url, width=300))

Сначала поработаю с текстовыми признаками с помощью bm25

---


$$
BM25(q, d) = \sum_{i=1}^{n} IDF(q_i) \times \frac{TF(q_i, d) \times (k_1 + 1)}{TF(q_i, d) + k_1 \times \left(1 - b + b \times \frac{|d|}{avgdl}\right)}
$$

$TF(q_i, d)$ - частота слова $q_i$ в документе

$avgdl$ - средняя длина документов

$$
IDF(q_i) = \log \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5}
$$

$n(q_i)$ - количество документов содержащих $q_i$.

$N$ - общее число документов.

---


bm25 это доработанная версия TF-IDF с помощью дополнительных коэфициентов мы отдаем больший приоритет коротким названиям и штрафуем за множественное повторение одного слова

In [ ]:
!pip install rank_bm25

In [25]:
import numpy as np
import string

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    return tokens

df['tokenized_name'] = df['name'].apply(preprocess_text)
df['tokenized_main_category'] = df['main_category'].apply(preprocess_text)
df['tokenized_sub_category'] = df['sub_category'].apply(preprocess_text)

display(df.head())

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category
0,Protoner 16Kg PVC Combo Home Gym Package,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71K-gcsnOY...,https://www.amazon.in/Protoner-16Kg-PVC-Combo-...,3.6,917,965.08,4999.0,NaN,"[protoner, 16kg, pvc, combo, home, gym, package]","[sports, fitness]","[strength, training]"
1,Unisex Hexagon Rubber Vinyl Fixed Dumbbell - P...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71pcrSXprs...,https://www.amazon.in/Generic-Vinyl-Coated-Iro...,4.4,367,389.00,6450.0,NaN,"[unisex, hexagon, rubber, vinyl, fixed, dumbbe...","[sports, fitness]","[strength, training]"
2,LAFILLETTE 150 lbs Metal Hand Gripper Forearm ...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41oTg0XNbF...,https://www.amazon.in/LALA-LIFE-Gripper-Streng...,4.5,381,489.00,1199.0,NaN,"[lafillette, 150, lbs, metal, hand, gripper, f...","[sports, fitness]","[strength, training]"
3,Aurion Genuine Leather Pro Weight Lifting Belt...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/813g2hRYuo...,https://www.amazon.in/Genuine-Comfortable-Adju...,4.4,446,799.00,899.0,NaN,"[aurion, genuine, leather, pro, weight, liftin...","[sports, fitness]","[strength, training]"
4,SPIRO PVC (Round/Hex) Dumbbells (1 Kg. X 2 = 2...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/61VcW8otph...,https://www.amazon.in/Dumbbells-Equipment-Fitn...,3.5,52,82.31,200.0,NaN,"[spiro, pvc, roundhex, dumbbells, 1, kg, x, 2,...","[sports, fitness]","[strength, training]"


In [26]:
!pip install rank_bm25

In [28]:
from rank_bm25 import BM25Okapi

tokenized_products = [t_p for t_p in df['tokenized_name']]
tokenized_main_category = [t_p for t_p in df['tokenized_main_category']]
tokenized_sub_category = [t_p for t_p in df['tokenized_sub_category']]

bm25_name = BM25Okapi(tokenized_products)
bm25_main_category = BM25Okapi(tokenized_main_category)
bm25_sub_category = BM25Okapi(tokenized_sub_category)

In [29]:
queries = [
    "smartphone realme new 8/256",
    "glue",
    "batterfly"
]

In [30]:
for query in queries:
    tokenized_query = preprocess_text(query)
    scores = 0.4 * bm25_name.get_scores(tokenized_query) + 0.3 * bm25_main_category.get_scores(tokenized_query) + 0.3 * bm25_sub_category.get_scores(tokenized_query)
    print(f"\nЗапрос: '{query}'")
    for (product, score, index) in sorted(zip(df['name'], scores, range(len(df))), key=lambda x: x[1], reverse=True)[:1]:
        print(f"  {product} -> BM25: {score:.3f}")
        url = df.iloc[index]['image']
        display(Image(url=url, width=300))


Запрос: 'smartphone realme new 8/256'
  Siwi Fast Charger for Realme 1, Realme 2, Realme 2 Pro, Realme 3, Realme 3 Pro, Realme 3i, Realme 4, Realme 5, Realme C1 2... -> BM25: 5.122



Запрос: 'glue'
  SOLDTRUE Huda Beauty Eye-Lash Glue False Eyelash Adhesive Glue Eyelashes Extension Glue 1pc -> BM25: 5.419



Запрос: 'batterfly'
  NiyatiFab Purple Color Dual Batterfly with Black Color Small Heart Combo Watch - for Girls -> BM25: 4.631


Сделаю синтетические данные для предобучения модели

In [34]:
def assign_relevance(row, query):
    score = 0
    tokenized_query = preprocess_text(query)

    text_score = 0.6 * bm25_name.get_scores(tokenized_query) + 0.3 * bm25_main_category.get_scores(tokenized_query) + 0.3 * bm25_sub_category.get_scores(tokenized_query)

    quality_score = 0.4 * (row['ratings'] / 5.0)

    total_score = text_score + quality_score

    if total_score >= 0.7:
        return 2  # Полностью релевантен
    elif total_score >= 0.3:
        return 1  # Частично релевантен
    else:
        return 0  # Не релевантен

In [39]:
query = "pasta bellisimo"

def assign_relevance(row, query, score_name, score_main_cat, score_sub_cat):
    tokenized_query = preprocess_text(query)

    text_score = (
        0.4 * score_name[row.name] +
        0.3 * score_main_cat[row.name] +
        0.3 * score_sub_cat[row.name]
    )

    quality_score = 0.4 * (row['ratings'] / 5.0)

    total_score = text_score + quality_score

    if total_score >= 0.7:
        return 2  # Полностью релевантен
    elif total_score >= 0.3:
        return 1  # Частично релевантен
    else:
        return 0  # Не релевантен

score_name = bm25_name.get_scores(tokenized_query)
score_main_cat = bm25_main_category.get_scores(tokenized_query)
score_sub_cat = bm25_sub_category.get_scores(tokenized_query)
df['relevance'] = df.apply(lambda row: assign_relevance(row, query, score_name, score_main_cat, score_sub_cat), axis=1)

df.sort_values('relevance', ascending=False).head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,relevance
1070052,NiyatiFab Purple Color Dual Batterfly with Bla...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/NiyatiFab-Arrival-Purple...,5.00,1,0.0,249.0,NaN,"[niyatifab, purple, color, dual, batterfly, wi...",[accessories],[watches],2
392108,ELECTROPRIME Men's Leather Briefs (Pack of 1) ...,men's clothing,Innerwear,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/ELECTROPRIME-Leather-Bri...,70.00,Usually dispatched in 4 to 5 weeks.,1790.0,2166.0,13355.0,"[electroprime, mens, leather, briefs, pack, of...","[mens, clothing]",[innerwear],2
385222,ELECTROPRIME 2pcs Sheer Pink Black Lace Underw...,men's clothing,Innerwear,https://m.media-amazon.com/images/I/51e-0slNpT...,https://www.amazon.in/ELECTROPRIME-Underwear-B...,70.00,Usually dispatched in 4 to 5 weeks.,1790.0,2166.0,6469.0,"[electroprime, 2pcs, sheer, pink, black, lace,...","[mens, clothing]",[innerwear],2
94314,Lengthy Handle Shoulder Bag - Hand Made Wire K...,accessories,Handbags & Clutches,https://m.media-amazon.com/images/I/41YbHBabOg...,https://www.amazon.in/Lengthy-handle-Shoulder-...,65.00,Usually dispatched in 6 to 7 days.,765.0,1200.0,NaN,"[lengthy, handle, shoulder, bag, hand, made, w...",[accessories],"[handbags, clutches]",2
918024,Sagwan Row in Brown Color by Artelier,women's shoes,Ballerinas,https://m.media-amazon.com/images/I/61+HIy4jk1...,https://www.amazon.in/Sagwan-Row-Brown-Color-A...,68.99,Only 1 left in stock.,0.0,5123.0,NaN,"[sagwan, row, in, brown, color, by, artelier]","[womens, shoes]",[ballerinas],2


Сгенерировал 100 синтетических запросов

In [47]:
all_search_queries = [
    # Electronics (25)
    "wireless headphones", "bluetooth earbuds", "smartphone android", "apple iphone",
    "smart watch", "fitness tracker", "bluetooth speaker", "portable power bank",
    "usb-c charger", "tablet case", "gaming mouse", "mechanical keyboard", "4k monitor",
    "laptop for work", "drawing tablet", "e-book reader", "smart watch with ECG",
    "wireless charger", "bluetooth headset", "streaming webcam", "recording microphone",
    "portable SSD", "wi-fi router", "smart speaker", "VR headset",

    # Home & Kitchen (20)
    "coffee maker", "air fryer", "instant pot", "kitchen blender", "stand mixer",
    "food processor", "toaster oven", "microwave oven", "vacuum cleaner", "air purifier",
    "humidifier", "essential oil diffuser", "bed sheet set", "memory foam pillow",
    "weighted blanket", "cookware set", "kitchen knives", "non-stick pan", "baking sheets",
    "food storage containers",

    # Fashion & Clothing (15)
    "running shoes", "sneakers for men", "women's dress", "jeans for women", "winter jacket",
    "rain coat", "athletic shorts", "yoga pants", "business shirt", "casual t-shirt",
    "wool socks", "swimwear", "handbag for women", "backpack for laptop", "sunglasses polarized",

    # Books & Media (15)
    "science fiction books", "romance novels", "business books", "cookbook for beginners",
    "children's story books", "self-help books", "programming textbooks", "fantasy book series",
    "mystery thriller", "audio books subscription", "kindle unlimited", "coloring books for adults",
    "educational books for kids", "biography of entrepreneurs", "graphic novels",

    # Toys & Games (10)
    "lego sets for kids", "board games for family", "educational toys", "video games ps5",
    "nintendo switch games", "outdoor play equipment", "puzzle games", "action figures",
    "stem learning toys", "baby toys",

    # Sports & Outdoors (10)
    "yoga mat", "dumbbell set", "exercise bike", "camping tent", "hiking backpack",
    "fishing rod", "bicycle for adults", "tennis racket", "basketball shoes", "water bottle insulated",

    # Beauty & Personal Care (5)
    "skincare set", "hair dryer", "electric toothbrush", "perfume for women", "makeup brush set"
]

Создам простенький класификатор вход 4 float (скоры по имени и категориям и рейтинг)

In [40]:
from torch import nn

model = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 3)
)

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [43]:
def assign_relevance_batch(scores_name, scores_main_cat, scores_sub_cat, ratings):
    text_scores = 0.6 * scores_name + 0.3 * scores_main_cat + 0.3 * scores_sub_cat
    quality_scores = 0.4 * (ratings / 5.0)
    total_scores = text_scores + quality_scores

    conditions = [
        total_scores >= 0.7,
        (total_scores >= 0.3) & (total_scores < 0.7),
        total_scores < 0.3
    ]
    choices = [2, 1, 0]  # 2=Полностью релевантен, 1=Частично, 0=Не релевантен

    return np.select(conditions, choices)

In [50]:
def prepare_data(df, queries):
    all_features = []
    all_labels = []
    tokenized_names = [preprocess_text(name) for name in df['name']]
    tokenized_main_cat = [preprocess_text(cat) for cat in df['main_category']]
    tokenized_sub_cat = [preprocess_text(cat) for cat in df['sub_category']]

    bm25_name = BM25Okapi(tokenized_names)
    bm25_main_category = BM25Okapi(tokenized_main_cat)
    bm25_sub_category = BM25Okapi(tokenized_sub_cat)

    for query in queries:
        tokenized_query = preprocess_text(query)

        scores_name = bm25_name.get_scores(tokenized_query)
        scores_main_cat = bm25_main_category.get_scores(tokenized_query)
        scores_sub_cat = bm25_sub_category.get_scores(tokenized_query)
        ratings = df['ratings'].values

        features = np.column_stack([
            scores_name,           # score_name
            scores_main_cat,       # score_main_cat
            scores_sub_cat,        # score_sub_cat
            ratings               # rating
        ])

        labels = assign_relevance_batch(scores_name, scores_main_cat, scores_sub_cat, ratings)

        all_features.append(features)
        all_labels.append(labels)

    X = np.vstack(all_features)
    y = np.hstack(all_labels)

    return X, y

In [45]:
def create_dataloaders(X, y, batch_size=32, train_ratio=0.7, val_ratio=0.15):
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.LongTensor(y)

    dataset = TensorDataset(X_tensor, y_tensor)

    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size]
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [54]:
def train_model(model, train_loader, val_loader, num_epochs=100, learning_rate=1e-4):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_y.size(0)
            train_correct += (predicted == batch_y).sum().item()

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += batch_y.size(0)
                val_correct += (predicted == batch_y).sum().item()

        train_loss_avg = train_loss / len(train_loader)
        val_loss_avg = val_loss / len(val_loader)
        train_accuracy = train_correct / train_total
        val_accuracy = val_correct / val_total

        train_losses.append(train_loss_avg)
        val_losses.append(val_loss_avg)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Train Loss: {train_loss_avg:.4f}, Train Acc: {train_accuracy:.2f}%')
            print(f'Val Loss: {val_loss_avg:.4f}, Val Acc: {val_accuracy:.2f}%')
            print('---')

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }

In [ ]:
X, y = prepare_data(df, all_search_queries[:4])

train_loader, val_loader, test_loader = create_dataloaders(X, y)

In [55]:
history = train_model(model, train_loader, val_loader, num_epochs=1)

In [56]:
history

{'train_losses': [0.010442423087758005],
 'val_losses': [9.760777015006265e-09],
 'train_accuracies': [99.76068964892083],
 'val_accuracies': [100.0]}

In [66]:
def predict_relevance(model, query):
    tokenized_query = preprocess_text(query)

    scores_name = bm25_name.get_scores(tokenized_query)
    scores_main_cat = bm25_main_category.get_scores(tokenized_query)
    scores_sub_cat = bm25_sub_category.get_scores(tokenized_query)
    ratings = df['ratings'].values

    features = np.column_stack([scores_name, scores_main_cat, scores_sub_cat, ratings])

    model.eval()
    with torch.no_grad():
        features_tensor = torch.FloatTensor(features)
        outputs = model(features_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        _, predictions = torch.max(outputs, 1)

    result_df = df.copy()
    result_df['predicted_class'] = predictions.numpy()
    result_df['predicted_prob_class_0'] = probabilities.numpy()[:, 0]
    result_df['predicted_prob_class_1'] = probabilities.numpy()[:, 1]
    result_df['predicted_prob_class_2'] = probabilities.numpy()[:, 2]

    result_df['ranking_score'] = result_df['predicted_prob_class_2'] + result_df['predicted_prob_class_1'] - result_df['predicted_prob_class_0']

    return result_df.sort_values('ranking_score', ascending=False)

In [71]:
queries = ["watch", "fitness tracker", "bluetooth speaker"]

In [72]:
for query in queries:
    print(f"\nЗапрос: '{query}'")
    display(predict_relevance(model, query).head())


Запрос: 'watch'


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,relevance,predicted_class,predicted_prob_class_0,predicted_prob_class_1,predicted_prob_class_2,ranking_score
1101283,"LUNAGARIYA®, Protective Cover for LG AC Remote...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/LUNAGARIYA%C2%AE-Protect...,4.3,62,199.0,399.0,NaN,"[lunagariya®, protective, cover, for, lg, ac, ...","[tv, audio, cameras]","[all, electronics]",1,1,1.560759e-36,1.0,1.380010e-14,1.0
20,Rioff® Height Increase Pull Up Bar for ChinUps...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/51sTzHGHbL...,https://www.amazon.in/Rioff-Height-Increase-Ch...,4.3,422,549.0,1499.0,NaN,"[rioff®, height, increase, pull, up, bar, for,...","[sports, fitness]","[strength, training]",1,1,1.560759e-36,1.0,1.380010e-14,1.0
21,ALLYSON FITNESS 3 in 1 Heavy Duty Multifunctio...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41aZmhOEZm...,https://www.amazon.in/ALLYSON-FITNESS-Multifun...,3.9,293,2699.0,5499.0,NaN,"[allyson, fitness, 3, in, 1, heavy, duty, mult...","[sports, fitness]","[strength, training]",1,1,7.598289e-16,1.0,1.536809e-19,1.0
23,Aurion PVC Dumbbells - Blue (Set of Two) | Han...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/61BHd-i819...,https://www.amazon.in/Aurion-Matrix2-Plastic-D...,4.0,"20,520",325.0,599.0,NaN,"[aurion, pvc, dumbbells, blue, set, of, two, h...","[sports, fitness]","[strength, training]",1,1,5.115287e-21,1.0,2.660327e-18,1.0
24,Hykes Heavy Pull Up Resistance Bands for Worko...,sports & fitness,Strength Training,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Hykes-Resistance-Stretch...,5.0,17,579.0,1499.0,NaN,"[hykes, heavy, pull, up, resistance, bands, fo...","[sports, fitness]","[strength, training]",1,1,0.000000e+00,1.0,1.584923e-10,1.0



Запрос: 'fitness tracker'


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,relevance,predicted_class,predicted_prob_class_0,predicted_prob_class_1,predicted_prob_class_2,ranking_score
11,Aurion PVC Dumbbells - (1kg*2) (2kg*2) (3kg*2)...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71t8ANKkBz...,https://www.amazon.in/Aurion-Multicolor-Dumbbe...,4.0,495,899.0,999.0,NaN,"[aurion, pvc, dumbbells, 1kg2, 2kg2, 3kg2, set...","[sports, fitness]","[strength, training]",1,2,0.000000e+00,4.538355e-13,1.000000e+00,1.0
1102778,Pro by Khadim's Women Fitness Lace-Up Sneakers,women's shoes,Shoes,https://m.media-amazon.com/images/I/71r+olmul2...,https://www.amazon.in/Pro-Khadims-Fitness-Lace...,0.0,4,450.0,899.0,NaN,"[pro, by, khadims, women, fitness, laceup, sne...","[womens, shoes]",[shoes],0,2,0.000000e+00,3.897304e-26,1.000000e+00,1.0
21,ALLYSON FITNESS 3 in 1 Heavy Duty Multifunctio...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41aZmhOEZm...,https://www.amazon.in/ALLYSON-FITNESS-Multifun...,3.9,293,2699.0,5499.0,NaN,"[allyson, fitness, 3, in, 1, heavy, duty, mult...","[sports, fitness]","[strength, training]",1,2,0.000000e+00,6.362877e-12,1.000000e+00,1.0
447905,DOCAT Handheld Portable Bidet Spray Bottle for...,appliances,Kitchen & Home Appliances,https://m.media-amazon.com/images/I/71f1wDPH9r...,https://www.amazon.in/DOCAT-Handheld-Portable-...,4.4,51,379.0,999.0,584.0,"[docat, handheld, portable, bidet, spray, bott...",[appliances],"[kitchen, home, appliances]",1,1,7.478310e-41,1.000000e+00,1.400497e-13,1.0
447902,AGARO VIBE 500 ml Adult/Baby Humidifier for Ho...,appliances,Kitchen & Home Appliances,https://m.media-amazon.com/images/I/817NbOc1KR...,https://www.amazon.in/AGARO-Adult-Humidifier-B...,4.0,399,1699.0,5499.0,581.0,"[agaro, vibe, 500, ml, adultbaby, humidifier, ...",[appliances],"[kitchen, home, appliances]",1,1,5.115287e-21,1.000000e+00,2.660327e-18,1.0



Запрос: 'bluetooth speaker'


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name,tokenized_main_category,tokenized_sub_category,relevance,predicted_class,predicted_prob_class_0,predicted_prob_class_1,predicted_prob_class_2,ranking_score
1101297,Pragati Systems® Medium Grain 24x36 Inch 10 Oz...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Pragati-Systems-Stretche...,4.5,938,940.0,1120.0,NaN,"[pragati, systems®, medium, grain, 24x36, inch...","[tv, audio, cameras]","[all, electronics]",1,1,2.410233e-43,1.0,4.521016e-13,1.0
31,BULMARC's Resistance Band Pull Up Assist Bands...,sports & fitness,Strength Training,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/BULMARCs-Resistance-Work...,4.3,"1,351",1699.0,4670.0,NaN,"[bulmarcs, resistance, band, pull, up, assist,...","[sports, fitness]","[strength, training]",1,1,1.560759e-36,1.0,1.380010e-14,1.0
30,Ap Tummy Twister Useful for Figure Tone-up Wei...,sports & fitness,Strength Training,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Twister-Tone-up-Reductio...,3.9,971,316.0,999.0,NaN,"[ap, tummy, twister, useful, for, figure, tone...","[sports, fitness]","[strength, training]",1,1,7.598289e-16,1.0,1.536809e-19,1.0
29,FirstFit Double Wheel Ab Roller with Non-Slip ...,sports & fitness,Strength Training,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/FirstFit-Non-Slip-Handle...,4.0,189,249.0,999.0,NaN,"[firstfit, double, wheel, ab, roller, with, no...","[sports, fitness]","[strength, training]",1,1,5.115287e-21,1.0,2.660327e-18,1.0
28,Amazon Brand - Symactive Neoprene Coated Dumbb...,sports & fitness,Strength Training,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Amazon-Brand-Symactive-N...,4.3,8,1709.0,3300.0,NaN,"[amazon, brand, symactive, neoprene, coated, d...","[sports, fitness]","[strength, training]",1,1,1.560759e-36,1.0,1.380010e-14,1.0


пока результат не очень позже буду разбираться почему